In the name of Allah

In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType,StructField, StringType
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip "/content/drive/MyDrive/Colab Notebooks/Divar/DMC-phase1-DataSets-20211225T055615Z-001.zip" 

path = "/content/DMC-phase1-DataSets"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/MyDrive/Colab Notebooks/Divar/DMC-phase1-DataSets-20211225T055615Z-001.zip
replace DMC-phase1-DataSets/reject_reasons_info.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace DMC-phase1-DataSets/DMC-Phase1-Validation.parquet? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace DMC-phase1-DataSets/DMC-Train.parquet? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
spark = SparkSession.builder.appName('Divar').getOrCreate()

df = spark.read.parquet(path + '/DMC-Train.parquet')

df = df.drop("reject_reason_id")

df = df.withColumn("review_label", when(df.review_label == "accept","true")\
      .when(df.review_label == "reject","false"))

In [ ]:
df.show()

+--------------------+--------------------+------------+
|             post_id|           post_data|review_label|
+--------------------+--------------------+------------+
|cb000599-2ee2-42c...|{"body_status": "...|        true|
|12063741-6634-444...|{"body_status": "...|       false|
|81c93119-5c06-412...|{"body_status": "...|        true|
|b5a5bfa7-03be-408...|{"brand": "\u0633...|        true|
|3414e920-dfaf-44a...|{"body_status": "...|       false|
|10aa8a3b-e465-45c...|{"body_status": "...|       false|
|79f05181-ddfd-44b...|{"body_status": "...|        true|
|3845f74f-6731-44f...|{"body_status": "...|        true|
|6fdbdc90-dc85-49e...|{"body_status": "...|       false|
|c8ba52f7-7025-4f8...|{"body_status": "...|        true|
|58af8fd6-3ef9-483...|{"body_status": "...|        true|
|389f6987-8cb8-48a...|{"body_status": "...|        true|
|07e64f4a-9767-435...|{"brand": "\u067e...|        true|
|df968fd4-fb04-427...|{"body_status": "...|        true|
|301684eb-b556-482...|{"body_st

In [ ]:
df.head()

Row(post_id='cb000599-2ee2-42c1-9f0e-32cfeb940398', post_data='{"body_status": "witout-color", "brand": "\\u062a\\u06cc\\u0628\\u0627::Tiba", "brand_model": "Tiba Sedan SX", "category": "light", "color": "\\u0633\\u0641\\u06cc\\u062f", "description": "\\u0628\\u062f\\u0648\\u0646 \\u0631\\u0646\\u06af\\u060c\\u06a9\\u0645 \\u06a9\\u0627\\u0631\\u06a9\\u0631\\u062f", "document": "single-sheet", "gearbox": "manual", "new_price": 103000000, "post_type": "\\u0641\\u0631\\u0648\\u0634\\u06cc", "selling_type": "cash", "third_party_insurance_deadline": "5", "title": "\\u062a\\u06cc\\u0628\\u0627 \\u0635\\u0646\\u062f\\u0648\\u0642\\u200c\\u062f\\u0627\\u0631 SX\\u060c \\u0645\\u062f\\u0644 \\u06f1\\u06f3\\u06f9\\u06f7", "usage": 23000, "year": "\\u06f1\\u06f3\\u06f9\\u06f7"}', review_label='true')

In [ ]:
import json
newJson = '''{"body_status": "witout-color", "brand": "\\u062a\\u06cc\\u0628\\u0627::Tiba", "brand_model": "Tiba Sedan SX", "category": "light", "color": "\\u0633\\u0641\\u06cc\\u062f", "description": "\\u0628\\u062f\\u0648\\u0646 \\u0631\\u0646\\u06af\\u060c\\u06a9\\u0645 \\u06a9\\u0627\\u0631\\u06a9\\u0631\\u062f", "document": "single-sheet", "gearbox": "manual", "new_price": 103000000, "post_type": "\\u0641\\u0631\\u0648\\u0634\\u06cc", "selling_type": "cash", "third_party_insurance_deadline": "5", "title": "\\u062a\\u06cc\\u0628\\u0627 \\u0635\\u0646\\u062f\\u0648\\u0642\\u200c\\u062f\\u0627\\u0631 SX\\u060c \\u0645\\u062f\\u0644 \\u06f1\\u06f3\\u06f9\\u06f7", "usage": 23000, "year": "\\u06f1\\u06f3\\u06f9\\u06f7"}'''
pdf = json. loads(newJson)
json. loads(newJson)

{'body_status': 'witout-color',
 'brand': 'تیبا::Tiba',
 'brand_model': 'Tiba Sedan SX',
 'category': 'light',
 'color': 'سفید',
 'description': 'بدون رنگ،کم کارکرد',
 'document': 'single-sheet',
 'gearbox': 'manual',
 'new_price': 103000000,
 'post_type': 'فروشی',
 'selling_type': 'cash',
 'third_party_insurance_deadline': '5',
 'title': 'تیبا صندوق\u200cدار SX، مدل ۱۳۹۷',
 'usage': 23000,
 'year': '۱۳۹۷'}

In [ ]:
def feature(string):
  pdf = json. loads(string)
  return pdf

In [ ]:
schema = StructType([ 
    StructField("body_status",StringType(),True), 
    StructField("brand",StringType(),True), 
    StructField("brand_model",StringType(),True), 
    StructField("category",StringType(),True), 
    StructField("color",StringType(),True), 
    StructField("description",StringType(),True), 
    StructField("document",StringType(),True), 
    StructField("gearbox",StringType(),True), 
    StructField("new_price",StringType(),True), 
    StructField("post_type",StringType(),True), 
    StructField("selling_type",StringType(),True), 
    StructField("third_party_insurance_deadline",StringType(),True), 
    StructField("title",StringType(),True), 
    StructField("usage", StringType(), True),
    StructField("year", StringType(), True)
  ])

df = df.withColumn("post_data", from_json("post_data", schema))\
    .select(col('post_id'),
            col('review_label'),
            col('post_data.*'))

In [ ]:
df.printSchema()

root
 |-- post_id: string (nullable = true)
 |-- review_label: string (nullable = true)
 |-- body_status: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- brand_model: string (nullable = true)
 |-- category: string (nullable = true)
 |-- color: string (nullable = true)
 |-- description: string (nullable = true)
 |-- document: string (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- new_price: string (nullable = true)
 |-- post_type: string (nullable = true)
 |-- selling_type: string (nullable = true)
 |-- third_party_insurance_deadline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- usage: string (nullable = true)
 |-- year: string (nullable = true)



In [ ]:
df.select(['third_party_insurance_deadline','usage','review_label','description']).show()

+------------------------------+------+------------+--------------------+
|third_party_insurance_deadline| usage|review_label|         description|
+------------------------------+------+------------+--------------------+
|                             5| 23000|        true|  بدون رنگ،کم کارکرد|
|                             1|120000|       false|اسفند 95\nبیمه بد...|
|                             8|  1400|        true|بیمه شخص ثالث،بیم...|
|                          null|     0|        true|■نمایندگی رجبی کد...|
|                             7|   800|       false|ماشین صفر برج ۹سا...|
|                          null|     0|       false|ماشین صفر هست و ت...|
|                            12|     0|        true|با سلام پژو207 پا...|
|                             1|220000|        true|۳تیکه عقب رنگ کاپ...|
|                             4|123456|       false|باسلام \nموتور هو...|
|                             1|   300|        true|ماشین کاملا کم کا...|
|                             1| 79000

In [ ]:
spark.stop()